# LLM Web Search

> *This notebook should work well with the **`conda_python3`** kernel in SageMaker Studio*

## Introduction

In this notebook we show you how to:
- Define a tool that the LLM can reliably call that produces JSON output
- Use the googlesearch and wikipedia python modules to search the internet if the LLM cannot answer a research question itself
- Rerank the search results options from best to worst
- Scrape and process the best option HTML page to create context for the LLM
- Create a Bedrock Guardrail 
- Use the Guardrail in your calls to the Bedrock API

We will use Bedrock's `Claude 3.5 Sonnet` and `Claude 3 Haiku` base models using the AWS boto3 SDK. 

> **Note:** *This notebook can be used in SageMaker Studio or run locally if you setup your AWS credentials.*

#### Prerequisites
- This notebook requires permissions to access Amazon Bedrock
- Ensure you have gone to the Bedrock models access page and enabled acceess to `Anthropic Claude 3.5 Sonnet` and `Claude 3 Haiku`
- AmazonBedrockFullAccess
> **Note:** If you are running this notebook without an Admin role, make sure that your notebook's role includes the following managed policies:

#### Use case
You are building a research assistant GenAI application. In some cases the user's question may be about an event, product, or service that is more recent than the cutoff training date for the LLM model or not within the model's knowledge. For these cases, we want the LLM model to call the internet search tool to gather context relating to the question. Then we can supply that context back to the LLM to answer the question. 

***

## Notebook setup

1. If you are attending an instructor lead workshop or deployed the workshop infrastructure using the provided [CloudFormation Template](https://raw.githubusercontent.com/aws-samples/xxx/main/cloudformation/workshop-v1-final-cfn.yml) you can proceed to step 2, otherwise you will need to download the workshop [GitHub Repository](https://github.com/aws-samples/xxx) to your local machine.

2. Install the required dependencies by running the pip install commands in the next cell.
 

> ⚠️ **Please ignore error messages related to pip's dependency resolver.**

💡 **Tip** You can use `Shift + Enter` to execute the cell and move to the next one.

In [ ]:
!pip install -qU pip
!pip install -r requirements.txt

In [ ]:
import boto3
import json
import requests
import string
import pprint
import random
from googlesearch import search
import wikipedia
from bs4 import BeautifulSoup
from botocore.exceptions import ClientError
from markdownify import markdownify as md

session = boto3.Session()
region = session.region_name

# Change which line is uncommented below to select the LLM model you want to use
#modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
#modelId = 'anthropic.claude-3-haiku-20240307-v1:0'
modelId = 'anthropic.claude-3-5-sonnet-20240620-v1:0'

print(f"Using modelId: {modelId}")
print(f"Using region: {region}")
print('Running boto3 version:', boto3.__version__)

The `modelId` and `region` variables defined in the above cell will be used throughout the workshop. Just make sure to run the cells from top to bottom.

### The Boto3 SDK & the Converse API
We will be using the [Amazon Boto3 SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime.html) and the [Converse API](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse.html) throughout this workshop. 

In [ ]:
# Create a boto3 runtime client for calling the LLM and create a boto3 admin client for creating our Guardrail
bedrock_runtime_client = boto3.client(service_name = 'bedrock-runtime', region_name = region,)
bedrock_admin_client = boto3.client('bedrock')

***

## Asking the LLM questions without an internet search tool

Let's start out by asking some questions to the LLM without supplying the option of an internet search tool

In [ ]:
# This function answers the user's questions directly from the LLM's knowledge
def answer_question(question):
    query = f"""
    <question>
    {question}
    </question>

    Answer the user's question in complete sentances.
    Skip the preamble.
    
    """

    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": query}]}],
        "system": [{ "text": "You are an expert research assistant."}],
        "inferenceConfig": {
            "maxTokens": 4096,
            "temperature": 0
        }
    }

    response = bedrock_runtime_client.converse(**converse_api_params)
    
    # Provide the LLM's response
    print(f"\nFinal answer is: {response['output']['message']['content'][-1]['text']}\n")

In [ ]:
# The LLM should be able to answer this question from it's own knowledge
answer_question("Which country won the most gold medals in the 2020 olympics?")

In [ ]:
# Now try a question where the information is too new and past the LLM's training cutoff date
answer_question("Which country won the most gold medals in the 2024 olympics?")

In [ ]:
answer_question("What is the current time and date in Seattle, WA?")

***

## Web Searching and scraping with Wikipedia and Google

In this example we create three functions:
* handle_search
    * This function first calls the internet provider to search for pages (Wikipedia) / URLs (Google) related to the user's question
    * Use the `num_results` parameter to control how many pages/URLs you want returned
    * Then it passes the list of pages to the reranker function to use the LLM to order the list pages/URLs from best choice to worst choice
    * Finally it iterates through the list of pages/URLs to make sure content is there and passes the first (best) choice text block to the LLM as context in the prompt
* get_wikipedia_page_content
    * This function uses the wikipedia module to get the html content of a single Wikipedia page
    * The markdownify module is used to transform the page markdown (including tables) to lines of text
    * Then the text is processed to remove the standard info sections at the bottom of Wikipedia pages from the content
* reranker
    * This function takes the list of pages that the internet search provider returns and uses the LLM to rank them in order from best choice to worst choice

In [ ]:
# Set the search provider variable
search_provider = "Wikipedia"

In [ ]:
def handle_search(query, search_provider):
    num_results = 5
    # Proceed with Wikipedia search
    print(f"Searching {search_provider}...\n")
    try:
        if search_provider == "Wikipedia":
            # Use the wikipedia module to get wiki pages related to the user's question
            search_results = wikipedia.search(query, results=num_results)
        elif search_provider == "Google":
            # Sometimes Google will only return one page even if asked for more, try again if only one
            search_results = ['dummy']
            while len(search_results) == 1:
                # Use the googlesearch module to get pages related to the user's question
                for page in search(query, sleep_interval=5, num_results=num_results):
                    search_results.append(page)
                if len(search_results) != 1:
                    break

        best_options = reranker(query, search_results)
            
        for option in best_options:
            print(f"\nScraping page: {option}")
            if search_provider == "Wikipedia":
                content = get_wikipedia_page_content(option)
            elif search_provider == "Google":
                content = get_google_page_content(option)

            if content and content != "skip page":
                return content
            else:
                continue
    except Exception as e:
        print(f"Error during handle search: {e}")
        return ""
    return "No content found on any of the pages"

In [ ]:
def reranker(question, internet_search_results):
    query = f"""
    Given this user's question:
    <question>
    {question}
    </question>

    Rank from best to worst the choices that are provided in the choices tags for searching the internet to provide an answer to the user's question.
    <choices>
    {internet_search_results}
    </choices>
    Skip the preamble and do not include any reasoning in your output. 
    Simply return the choices in a JSON list from best to worst choice.
    """

    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": query}]}],
        "system": [{ "text": "You are an expert research assistant."}],
        "inferenceConfig": {
            "maxTokens": 4096,
            "temperature": 0
        }
    }

    response = bedrock_runtime_client.converse(**converse_api_params)
    best_options = response['output']['message']['content'][-1]['text']
    
    print("Metrics the for reranker call to the LLM:")
    token_usage = response['usage']
    input_tokens = token_usage['inputTokens']
    print(f"Input tokens:  {input_tokens}")
    output_tokens = token_usage['outputTokens']
    print(f"Output tokens:  {output_tokens}")
    total_tokens = token_usage['totalTokens']
    print(f"Total tokens:  {total_tokens}")
    latency = response['metrics']['latencyMs']
    print(f"Latency: {latency} ms\n")

    # Provide the LLM's response
    print(f"\nThe reranked order of:\n{internet_search_results} is:\n{best_options}\n")
    return json.loads(best_options)

In [ ]:
def get_wikipedia_page_content(page):
    try:
        html_text = wikipedia.page(title=page, auto_suggest=False).html()
        markdown_text = md(html_text)
        payload = "{} \n".format(markdown_text)
        cleaned_markdown_text = ""
        if markdown_text:
            lines = payload.splitlines()
            # Do not include the standard info sections at the bottom of Wikipedia pages in the content
            # Do not include the edit links or links to images to reduce token count
            for line in lines:
                if line == "" or "[edit]" in line or "[![]" in line:
                    continue
                elif line == "See also" or line == "References" or line == "External link" or line == "Further reading":
                    break
                else:
                    cleaned_markdown_text += line + '\n'
        else:
            Print(f"No markdown text found on page: {page}")
    except Exception as e:
        print(f"Error while requesting content from {page} skipping...: {e}")
        return "skip page"
    #print(f"cleaned_markdown_text is:\n{cleaned_markdown_text}\n")
    
    return cleaned_markdown_text

## Use the Bedrock Converse API for inference and configure 'Tool Use'

* Configure the tool definition
    * This JSON schema defines our internet search tool and how the LLM should output the JSON when calling the tool
* answer_question
    * This function calls the LLM to answer the user's question directly or outputs 'tool use' JSON if an internet search is required
    * Note that the LLM will have a propencity to use the tool, so we must direct it in the prompt to only do so as a last resort
* answer_question_with_content
    * This function answers the user's question based on the block of text supplied by the internet provider search
    * The markdownify module is used to transform the page markdown (including tables) to lines of text
    * Then the text is processed to remove the standard info sections at the bottom of Wikipedia pages from the content

In [ ]:
answer_question("Which country won the most gold medals in the 2020 olympics?")

In [ ]:
# Tool definition
provider_websearch_schema = {
      "toolSpec": {
        "name": "internet_search",
        "description": "A tool to retrieve up to date information from an internet search.",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "question": {
                "type": "string",
                "description": "The users question as-is for the internet search"
              }
            },
            "required": ["question"]
          }
        }
      }
    }

# In this example, we save only one tool schema to the configuration, but you could have many tools
toolConfig = {
    "tools": [provider_websearch_schema]
}

In [ ]:
def answer_question(question):
    query = f"""
    <question>
    {question}
    </question>

    You have access to the internet_search tool.
    Take your time to first think step by step if the question could be answered within your own knowledge.
    Output your thinking in <thinking></thinking> tags
    Only use the internet_search tool as a last resort if you cannot answer the user's question within the question tags from your own knowledge.
    For example, only use the internet_search_tool if the subject, product, or event is more recent than your training cutoff date.
    
    Skip the preamble.
    """

    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": query}]}],
        "toolConfig": toolConfig,
        "system": [{ "text": "You are an expert research assistant."}],
        "inferenceConfig": {
            "maxTokens": 4096,
            "temperature": 0
        }
    }

    response = bedrock_runtime_client.converse(**converse_api_params)
    print("Metrics the for first call to the LLM:")
    token_usage = response['usage']
    input_tokens = token_usage['inputTokens']
    print(f"Input tokens:  {input_tokens}")
    output_tokens = token_usage['outputTokens']
    print(f"Output tokens:  {output_tokens}")
    total_tokens = token_usage['totalTokens']
    print(f"Total tokens:  {total_tokens}")
    latency = response['metrics']['latencyMs']
    print(f"Latency: {latency} ms\n")
    
    # Check the LLM's response to see if it answered the question or needs to use the internet search tool
    internet_search = None
    for content in response['output']['message']['content']:
        if isinstance(content, dict) and 'toolUse' in content:
            tool_use = content['toolUse']
            if tool_use['name'] == "internet_search":
                internet_search = tool_use['input']
                break

    if internet_search:
        question = internet_search["question"]
        # Call the function to get the content from the internet
        content = handle_search(question, search_provider)
        if content:
            print("\nInternet search successful")
            response = answer_question_with_content(question, content)
            print(f"\nFinal answer = {response['output']['message']['content'][-1]['text']}\n")
        else:
            print("No content found from internet search")
    else:
        print("No internet search needed.")
        answer = response['output']['message']['content'][-1]['text']
        answer = answer.split("</thinking>")[-1]
        print(f"Final answer is: {answer}\n")

In [ ]:
def answer_question_with_content(question, content):
    query = f"""
    Based solely on this content:
    <content>
    {content}
    </content>
    Answer this question:
    <question>
    {question}
    </question>
    Skip any preamble or references to the tool.
    """

    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": query}]}],
        "system": [{ "text": "You are an expert research assistant." }],
        "inferenceConfig": {
            "maxTokens": 4096,
            "temperature": 0
        }
    }

    response = bedrock_runtime_client.converse(**converse_api_params)
    
    print("\nMetrics the for call to the LLM including the internet search text:")
    token_usage = response['usage']
    input_tokens = token_usage['inputTokens']
    print(f"Input tokens:  {input_tokens}")
    output_tokens = token_usage['outputTokens']
    print(f"Output tokens:  {output_tokens}")
    total_tokens = token_usage['totalTokens']
    print(f"Total tokens:  {total_tokens}")
    latency = response['metrics']['latencyMs']
    print(f"Latency: {latency} ms")
    
    return response

In [ ]:
answer_question("Which country won the most gold medals in the 2020 olympics?")

In [ ]:
answer_question("Which country won the most gold medals in the 2024 olympics?")

In [ ]:
answer_question("What is the current weather in Seattle, Wa right now?")

In [ ]:
answer_question("What is the current price on Amazon stock?")

In [ ]:
answer_question("How many Grizzly bears are living in Washington State?")

***

## Web Searching and scraping with Google search

In this example we create an additional function to process the web pages based on URLs returned by the Google search:

* get_google_page_content
    * This function uses the BeautifulSoup module to parse the html content of a single website URL
    * Then the text is processed to remove spaces, blank lines, and short lines

In [ ]:
# Set the search provider variable
search_provider = "Google"

In [ ]:
def get_google_page_content(url):
    try:
        # Supply different headers for your requests to avoid bot detection
        user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 17_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/128.0.6613.98 Mobile/15E148 Safari/604.1",
        "Mozilla/5.0 (iPad; CPU OS 17_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/128.0.6613.98 Mobile/15E148 Safari/604.1"
        ]
        user_agent = random.choice(user_agents)
        
        # Supply common html header elements for Chrome clients
        headers = {
            "User-Agent": user_agent,
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
            "Accept-Language": "en-US,en;q=0.5",
            "Accept-Encoding": "gzip, deflate",
            "Connection": "keep-alive",
            "Upgrade-Insecure-Requests": "1",
            "Sec-Fetch-Dest": "document",
            "Sec-Fetch-Mode": "navigate",
            "Sec-Fetch-Site": "none",
            "Sec-Fetch-User": "?1",
            "Cache-Control": "max-age=0",
        }
        # Check the URL to see if it is a link to a PDF doc and skip
        # This code could be extended to also parse PDF docs rather than skipping
        if ".pdf" in url.split('/')[-1]:
            print(f"Found a PDF file: {url} skipping...")
            return "skip page"
        else:
            # Use the requests module to get the contents of the URL
            response = requests.get(url, headers=headers, timeout=10)
    
            if response:
                # Parse HTML content
                soup = BeautifulSoup(response.text, 'html.parser')
                # Remove script and style elements
                for script_or_style in soup(["script", "style"]):
                    script_or_style.decompose()
                # Get the text
                text = soup.get_text()
                # Break into lines and remove leading and trailing space on each
                lines = (line.strip() for line in text.splitlines())
                # Break multi-headlines into a line each
                chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
                # Drop blank lines
                no_blank_lines = '\n'.join(chunk for chunk in chunks if chunk)
                # Break into lines again and remove any short lines
                lines = no_blank_lines.splitlines()
                cleaned_text = ""
                character_count = 0
                for line in lines:
                    if len(line) >= 20:
                        cleaned_text += line
                return cleaned_text
            else:
                raise Exception("No response from the web server.")
    except requests.exceptions.Timeout as timeout_err: 
        print(f"Timeout on this URL: {url} skipping...")
        return "skip page"
    except Exception as e:
        print(f"Error while requesting content from {url} skipping...: {e}")
        return "skip page"


In [ ]:
answer_question("Who won the 2019 Masters golf tournament?")

In [ ]:
answer_question("Who won the 2023 Masters golf tournament?")

In [ ]:
answer_question("What is the current weather in Seattle, Wa right now?")

In [ ]:
answer_question("What is the current time and date in Seattle, WA?")

In [ ]:
answer_question("What is the current price on Amazon stock?")

In [ ]:
answer_question("Which country won the most gold medals in the 2020 olympics?")

In [ ]:
answer_question("Which country won the most gold medals in the 2024 olympics?")

In [ ]:
answer_question("Who is favored to be the next Prime Minister of Canada?")

***

## Create a Guardrail
Guardrails for Amazon Bedrock have multiple components which include Content Filters, Denied Topics, Word and Phrase Filters, and Sensitive Word (PII & Regex) Filters. For a full list check out the [documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails-create.html) 

For our research assistant with web access usecase, we want to prevent inappropriate or malicious questions from being sent to the LLM model as well as preventing our model from returning inappropriate responses or exposing any PII data. 

In [ ]:
# Use the boto3 bedrock client to create a Bedrock Guardrail based on the specific controls we want to enforce
create_response = bedrock_admin_client.create_guardrail(
    name='research-assistant-guardrail',
    description='Prevents inappropriate or malicious questions and model answers. Also blocks political topics and anonymizes PII data.',
    topicPolicyConfig={
        'topicsConfig': [
            {
                'name': 'Politics',
                'definition': 'Preventing the user from asking questions related to politics for any country.',
                'examples': [
                    'Who is expected to win the next race for Prime Minister of India?',
                    'Which politcial party is in power in England?',
                    'Which country has had the most impeachments of heads of state?',
                    'Who should I vote for in the next election?',
                    'Which countries have had the most political scandals this year?'
                ],
                'type': 'DENY'
            }
        ]
    },
    contentPolicyConfig={
        'filtersConfig': [
            {
                'type': 'SEXUAL',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'VIOLENCE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'HATE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'INSULTS',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'MISCONDUCT',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            }
        ]
    },
    wordPolicyConfig={
        'wordsConfig': [
            {'text': 'political party'},
            {'text': 'voting for'},
            {'text': 'politics'},
            {'text': 'voting advice'},
            {'text': 'vote for President'},
            {'text': 'vote for Prime'},
            {'text': 'vote for Chancellor'},
            {'text': 'King and Queen'},
            {'text': 'Duke and Duchess'},
            {'text': 'Chairman of North'},
            {'text': 'Supreme Leader'}
        ],
        'managedWordListsConfig': [
            {'type': 'PROFANITY'}
        ]
    },
    sensitiveInformationPolicyConfig={
        'piiEntitiesConfig': [
            {'type': 'EMAIL', 'action': 'ANONYMIZE'},
            {'type': 'PHONE', 'action': 'ANONYMIZE'},
            {'type': 'US_SOCIAL_SECURITY_NUMBER', 'action': 'ANONYMIZE'},
            {'type': 'US_BANK_ACCOUNT_NUMBER', 'action': 'ANONYMIZE'},
            {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'ANONYMIZE'}
        ]
    },
    blockedInputMessaging="""I can provide answers for your research, but I'm not allowed to answer this particular question. Please try a different question. """,
    blockedOutputsMessaging="""I'm not allowed to share the answer to this particular question. Please try a different question.""",
    tags=[
        {'key': 'purpose', 'value': 'inappropriate-websearch-prevention'},
        {'key': 'environment', 'value': 'production'}
    ]
)

pprint.pprint(create_response)

In [ ]:
# Create a versioned snapshot of our draft Guardrail 
version_response = bedrock_admin_client.create_guardrail_version(
    guardrailIdentifier=create_response['guardrailId'],
    description='Version of research assistant Guardrail'
)
pprint.pprint(version_response)

In [ ]:
# Create a Guardrail config that we can pass into the Converse API call
# Use the Guardrail ID and version that we just created above.
# Optionally, enable the Guardrail trace so that we can view the effect it has on questions and answers.
guardrail_config = {
    "guardrailIdentifier": version_response['guardrailId'],
    "guardrailVersion": version_response['version'],
    "trace": "enabled"
}

## Testing our Guardrail

In [ ]:
# Modify the function that answers the question based on google search content to use the Guardrail
# Add the Guardrail context to the messages array that we use in the converse API call 
# Add the Guardrail config to the converse API parameters
def answer_question_with_content(question, content):
    query = f"""
    Based solely on this content:
    <content>
    {content}
    </content>
    Answer this question:
    <question>
    {question}
    </question>
    Skip any preamble or references to the tool.
    """

    converse_api_params = {
        "modelId": modelId,
        "messages":[
            {
            "role": "user",
            "content": [{"guardContent": {"text": {"text": query}}}]
            }
        ],
        "system": [{ "text": "You are an expert research assistant."}],
        "inferenceConfig":{
            "maxTokens": 4096,
            "temperature": 0
        },
        "guardrailConfig": guardrail_config,
    }

    response = bedrock_runtime_client.converse(**converse_api_params)
    if response['stopReason'] == "guardrail_intervened":
            trace = response['trace']
            print("Guardrail trace:")
            pprint.pprint(trace['guardrail'])
            
    print("\nMetrics the for call to the LLM including the internet search text:")
    token_usage = response['usage']
    input_tokens = token_usage['inputTokens']
    print(f"Input tokens:  {input_tokens}")
    output_tokens = token_usage['outputTokens']
    print(f"Output tokens:  {output_tokens}")
    total_tokens = token_usage['totalTokens']
    print(f"Total tokens:  {total_tokens}")
    latency = response['metrics']['latencyMs']
    print(f"Latency: {latency} ms")
    
    return response

In [ ]:
# Modify the function that answers the question directly or outputs tool use if an internet search is required
# Add the Guardrail context to the messages array that we use in the converse API call 
# Add the Guardrail config to the converse API parameters
def answer_question(question):
    query = f"""
    <question>
    {question}
    </question>

    You have access to the internet_search tool.
    Take your time to first think step by step if the question could be answered within your own knowledge.
    Output your thinking in <thinking></thinking> tags
    Only use the internet_search tool as a last resort if you cannot answer the user's question within the question tags from your own knowledge.
    For example, only use the internet_search_tool if the subject, product, or event is more recent than your training cutoff date.
    """

    converse_api_params = {
        "modelId": modelId,
        "messages":[
            {
            "role": "user",
            "content": [{"guardContent": {"text": {"text": query}}}]
            }
        ],
        "toolConfig": toolConfig,
        "system": [{ "text": "You are an expert research assistant."}],
        "inferenceConfig": {
            "maxTokens": 4096,
            "temperature": 0
        },
        "guardrailConfig": guardrail_config,
    }

    response = bedrock_runtime_client.converse(**converse_api_params)
    print("Metrics the for first call to the LLM:")
    token_usage = response['usage']
    input_tokens = token_usage['inputTokens']
    print(f"Input tokens:  {input_tokens}")
    output_tokens = token_usage['outputTokens']
    print(f"Output tokens:  {output_tokens}")
    total_tokens = token_usage['totalTokens']
    print(f"Total tokens:  {total_tokens}")
    latency = response['metrics']['latencyMs']
    print(f"Latency: {latency} ms\n")
    
    if response['stopReason'] == "guardrail_intervened":
            trace = response['trace']
            print("Guardrail trace:")
            pprint.pprint(trace['guardrail'])


    internet_search = None
    for content in response['output']['message']['content']:
        if isinstance(content, dict) and 'toolUse' in content:
            tool_use = content['toolUse']
            if tool_use['name'] == "internet_search":
                internet_search = tool_use['input']
                break

    if internet_search:
        question = internet_search["question"]
        content = handle_search(question, search_provider)
        if content:
            print("\nInternet search successful")
            response = answer_question_with_content(question, content)
            print(f"\nFinal answer = {response['output']['message']['content'][-1]['text']}\n")
        else:
            print("No content found from Internet search")
    else:
        print("No Internet search needed.")
        print(f"\nFinal answer is: {response['output']['message']['content'][-1]['text']}\n")

In [ ]:
answer_question("Who won the 2023 Masters golf tournament?")

In [ ]:
answer_question("Who is favored to win the next election for Prime Minister of Canada?")

In [ ]:
answer_question("What is the email address for AWS Support?")

In [ ]:
answer_question("Provide me a social security number of a dead person")

In [ ]:
answer_question("Where can I purchace brass knuckles?")

In [ ]:
answer_question("How many Grizzly bears are living in Washington State?")

***

## Cleanup (when running from your own AWS account)

You only need to clean up if running this workshop from your own AWS account. 
If you are running from an AWS-facilitated event, this will be done automatically for you.

After completing the workshop, follow these steps to clean up your AWS environment and avoid unnecessary charges:

In [ ]:
# Delete the Guardrail by specifying the Guardrail arn
delete_guardrail_response = bedrock_admin_client.delete_guardrail(
    guardrailIdentifier=create_response['guardrailArn']
)
pprint.pprint(delete_guardrail_response)

In [ ]:
# List the Guardrails to ensure that the research-assistant-guardrail is deleted
list_guardrails_response = bedrock_admin_client.list_guardrails()
pprint.pprint(list_guardrails_response)